In [1]:
import os
import polars as pl
import numpy as np
import torch.nn as nn
from polars import DataFrame
import torch
from torch.utils.data import DataLoader
from typing import Tuple
import wandb

from path import RESULT_DIRECTORY
from develop import reload_function, reload_module

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [15]:
from data.source.pg_experiment import get_pg_experiment_dataframe

df_pron, _ = get_pg_experiment_dataframe(".ogg")
dataframe = df_pron.with_columns(word_id = pl.struct("word_id").rank("dense"))
dataframe = dataframe.filter(pl.col("stage") == 1)


N_WORDS = dataframe.select(pl.col("word_id").n_unique()).to_numpy()[0][0]
print(f"Number of unique words: {N_WORDS}")
print(f"Number of samples: {dataframe.shape[0]}")
print(f"Number of speakers: {dataframe.select(pl.col('id_student').n_unique()).to_numpy()[0][0]}")

Number of unique words: 12
Number of samples: 5855
Number of speakers: 547


In [3]:
def split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []
    
    rng = np.random.RandomState(seed)
    
    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)
        
        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)
        
        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])
    
    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)
    
    
    return train_df, val_df, test_df

In [4]:
from dataset import Cast, TorchDataset
from pytorch_dataloader import build_collate_fn, PaddingCollate, DefaultCollate
from transformation import Channels, RMSEnergy, TorchVadMFCC, ZeroCrossingRate

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = split(dataframe, label_col="word_id", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)

def to_dataset(dataframe: DataFrame) -> TorchDataset:
    return TorchDataset(
    Cast(dataframe.get_column("rec_path"), Channels("stack","multiply")(
            TorchVadMFCC(delta=0),
        )),
    Cast(dataframe.get_column("rec_path"), Channels("cat","multiply")(
            ZeroCrossingRate(),
            RMSEnergy(),
        )),
    Cast(dataframe.get_column("word_id"), lambda x: torch.tensor(x-1, dtype=torch.long)),
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=2),
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=1),
    DefaultCollate(),
    DefaultCollate(),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [ ]:
from os import name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#note, if you are using Windows you MUST set `num_workers=0` - TL;DT multithreading DON'T work in notebooks because Windows DON'T have `fork()`
num_workers = 0 if name == "nt" else 4
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)

for x in next(iter(train_loader)):
    print(x.shape)

In [ ]:
from pytorch_dataloader import MemoryLoadedDataLoader
train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")
test_loader = MemoryLoadedDataLoader(test_loader, device=device)

In [12]:
from models.FusionCNN import ContextFusionCNN, ContextCNN
from models.Guesser import ContextGuesser

cfcnn_model = ContextFusionCNN(n_2d_channels=1, n_1d_channels=2, num_words=N_WORDS, dropout_rate=0.5)
ccnn_model = ContextCNN(n_2d_channels=1, num_words=N_WORDS, dropout_rate=0.5)
guesser_model = ContextGuesser(num_words=N_WORDS)
models = [guesser_model, cfcnn_model, ccnn_model]

## Hyperparameters

In [8]:
from type_helpers import Kwargs

epochs = 140

Optimizer = torch.optim.Adam
OptimizerKwargs = Kwargs(Optimizer)
optimizer_config = OptimizerKwargs(
    lr=1e-4,
    weight_decay= 1e-4,
)

Scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau
SchedulerKwargs = Kwargs(Scheduler)

scheduler_config = SchedulerKwargs(
    mode='min',
    factor=0.5,
    patience=5
)
criterion = nn.BCELoss()


In [9]:
from models.SimpleCNN_v2 import train, evaluate

def train_model(model):
    model = model.to(device)
    optimizer = Optimizer(model.parameters(), **optimizer_config)
    scheduler = Scheduler(optimizer, **scheduler_config)

    run = wandb.init(
        # name of the run
        name=f"14+11 (0.5 dropout) experiment - {model.__class__.__name__}",
        config={
            "Name": f"{model.__class__.__name__}",
            "learning_rate": optimizer_config["lr"],
            "optimizer": f"{Optimizer.__name__}",
            "criterion": f"{criterion.__class__.__name__}",
            "architecture": f"{model.__class__.__name__}",
            "architecture_details": str(model),
            "dataset": "Stage-I",
            "train_val_test(%)": f'{TRAIN_SPLIT}-{VAL_SPLIT}-{TEST_SPLIT}',
            "epochs": epochs,
        },
    )

    # Training loop
    for epoch in range(epochs):
        for label_version in ["v2"]:
            train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device)
            scheduler.step(val_loss)
        run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
        print(
            f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    run.log({"model_eval": model.eval()})
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    run.log({"test_acc": test_acc, "test_loss": test_loss})
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    # Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
    torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, f"{model.__class__.__name__}.pth"))
    artifact = wandb.Artifact(f"{model.__class__.__name__}-model", type="model")
    artifact.add_file(os.path.join(RESULT_DIRECTORY, f"{model.__class__.__name__}.pth"))
    run.log_artifact(artifact)

    # Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
    run.finish()
    return test_acc


In [13]:
test_accuracies = {}
for model in models:
    test_acc = train_model(model)
    test_accuracies[model.__class__.__name__] = test_acc 

Epoch 1, Train Loss: 0.6832, Train Acc: 0.5673, Val Loss: 0.6747, Val Acc: 0.5867
Epoch 2, Train Loss: 0.6731, Train Acc: 0.5824, Val Loss: 0.6643, Val Acc: 0.5867
Epoch 3, Train Loss: 0.6622, Train Acc: 0.5861, Val Loss: 0.6550, Val Acc: 0.5867
Epoch 4, Train Loss: 0.6560, Train Acc: 0.5867, Val Loss: 0.6469, Val Acc: 0.5867
Epoch 5, Train Loss: 0.6477, Train Acc: 0.5869, Val Loss: 0.6395, Val Acc: 0.5867
Epoch 6, Train Loss: 0.6470, Train Acc: 0.5849, Val Loss: 0.6340, Val Acc: 0.5867
Epoch 7, Train Loss: 0.6437, Train Acc: 0.5906, Val Loss: 0.6298, Val Acc: 0.5867
Epoch 8, Train Loss: 0.6397, Train Acc: 0.5844, Val Loss: 0.6260, Val Acc: 0.5867
Epoch 9, Train Loss: 0.6382, Train Acc: 0.5981, Val Loss: 0.6228, Val Acc: 0.5867
Epoch 10, Train Loss: 0.6317, Train Acc: 0.6060, Val Loss: 0.6197, Val Acc: 0.6003
Epoch 11, Train Loss: 0.6352, Train Acc: 0.6032, Val Loss: 0.6179, Val Acc: 0.6217
Epoch 12, Train Loss: 0.6303, Train Acc: 0.6038, Val Loss: 0.6161, Val Acc: 0.6507
Epoch 13, Tra

test_acc,▁
test_loss,▁
train_acc,▁▁▁▁▂▃▄▆▆▇▇▆▇▆▇█▇▆▆▇▆▆█▇▆▇▇▆▆▇▆▇▇▇▇▇▇▇▆▆
train_loss,█▅▄▂▂▂▁▂▂▂▂▁▁▂▂▂▂▁▁▂▁▂▂▂▂▂▁▂▁▂▂▁▁▂▂▁▂▁▁▁
val_acc,▁▁▁▁▅███████████████████████████████████
val_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model_eval,ContextGuesser( (w...
test_acc,0.625
test_loss,0.63175
train_acc,0.63626
train_loss,0.62454


Epoch 1, Train Loss: 0.6888, Train Acc: 0.5291, Val Loss: 0.6791, Val Acc: 0.6234
Epoch 2, Train Loss: 0.6821, Train Acc: 0.5476, Val Loss: 0.6703, Val Acc: 0.6183
Epoch 3, Train Loss: 0.6759, Train Acc: 0.5716, Val Loss: 0.6625, Val Acc: 0.6379
Epoch 4, Train Loss: 0.6748, Train Acc: 0.5801, Val Loss: 0.6570, Val Acc: 0.6422
Epoch 5, Train Loss: 0.6641, Train Acc: 0.6049, Val Loss: 0.6481, Val Acc: 0.6405
Epoch 6, Train Loss: 0.6627, Train Acc: 0.6052, Val Loss: 0.6436, Val Acc: 0.6379
Epoch 7, Train Loss: 0.6587, Train Acc: 0.6003, Val Loss: 0.6367, Val Acc: 0.6396
Epoch 8, Train Loss: 0.6535, Train Acc: 0.6040, Val Loss: 0.6314, Val Acc: 0.6379
Epoch 9, Train Loss: 0.6492, Train Acc: 0.6183, Val Loss: 0.6265, Val Acc: 0.6422
Epoch 10, Train Loss: 0.6466, Train Acc: 0.6135, Val Loss: 0.6217, Val Acc: 0.6524
Epoch 11, Train Loss: 0.6410, Train Acc: 0.6194, Val Loss: 0.6179, Val Acc: 0.6584
Epoch 12, Train Loss: 0.6398, Train Acc: 0.6326, Val Loss: 0.6125, Val Acc: 0.6755
Epoch 13, Tra

test_acc,▁
test_loss,▁
train_acc,▁▂▃▃▄▄▅▅▆▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██▇▇█████▇██
train_loss,█▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁
val_acc,▁▁▁▄▅▅▆▅▅▅▆▆▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████
val_loss,█▇▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model_eval,ContextFusionCNN( ...
test_acc,0.70068
test_loss,0.57839
train_acc,0.7577
train_loss,0.5083


Epoch 1, Train Loss: 0.6780, Train Acc: 0.5901, Val Loss: 0.6737, Val Acc: 0.5875
Epoch 2, Train Loss: 0.6757, Train Acc: 0.5906, Val Loss: 0.6706, Val Acc: 0.5978
Epoch 3, Train Loss: 0.6728, Train Acc: 0.5995, Val Loss: 0.6655, Val Acc: 0.6080
Epoch 4, Train Loss: 0.6707, Train Acc: 0.5955, Val Loss: 0.6613, Val Acc: 0.6089
Epoch 5, Train Loss: 0.6682, Train Acc: 0.6038, Val Loss: 0.6577, Val Acc: 0.6123
Epoch 6, Train Loss: 0.6632, Train Acc: 0.6069, Val Loss: 0.6524, Val Acc: 0.6140
Epoch 7, Train Loss: 0.6534, Train Acc: 0.6174, Val Loss: 0.6435, Val Acc: 0.6149
Epoch 8, Train Loss: 0.6540, Train Acc: 0.6166, Val Loss: 0.6354, Val Acc: 0.6285
Epoch 9, Train Loss: 0.6516, Train Acc: 0.6106, Val Loss: 0.6325, Val Acc: 0.6354
Epoch 10, Train Loss: 0.6478, Train Acc: 0.6251, Val Loss: 0.6269, Val Acc: 0.6388
Epoch 11, Train Loss: 0.6473, Train Acc: 0.6280, Val Loss: 0.6235, Val Acc: 0.6379
Epoch 12, Train Loss: 0.6445, Train Acc: 0.6257, Val Loss: 0.6188, Val Acc: 0.6524
Epoch 13, Tra

test_acc,▁
test_loss,▁
train_acc,▁▂▂▁▂▂▃▃▃▄▄▄▄▅▅▅▅▆▅▆▅▆▆▆▆▆▆▇▇▇▇██▇████▇█
train_loss,███▇▇▆▆▅▅▅▅▅▅▄▅▄▄▄▄▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁
val_acc,▁▂▃▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█▇███████████
val_loss,█▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model_eval,ContextCNN( (featu...
test_acc,0.72109
test_loss,0.56043
train_acc,0.73803
train_loss,0.53015


In [14]:
for model_name, acc in test_accuracies.items():
    print(f"Model: {model_name}, Test Accuracy: {acc:.4f}")

Model: ContextGuesser, Test Accuracy: 0.6250
Model: ContextFusionCNN, Test Accuracy: 0.7007
Model: ContextCNN, Test Accuracy: 0.7211
